# T2

We would like to find the answer for T2 by using the K Nearest Neighbors algorithm. 
With this algorithm we will find top K papers which are most similar (the nearest neighbors) of a particular paper to cite. 
Afterwards, we plan to retrieve the authors of these top K papers, and find top K most frequent authors from this auhtor list.

For the feature computation, we plan to use the following relevant columns:
- Paper affiliations
- Author research interests
- Paper Year

The process:
1. Index all the of the categorical columns which we are going to use for our feature computation (columns: paper affiliations, author research interests)
2. Apply One Hot encoding to these columns
3. Create VectorAssembler for all of the accounted features
4. Afterwards, create the pipeline for data transformation (p.1-2) and vector assembling
5. Apply this pipeline to our dataset
6. Divide the data into training and test set by the years
7. Train the data using KNN algorithm from sklearn

In [1]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import *

In [2]:
from helpers import createDFFromFileAndSchema

In [3]:
spark=SparkSession.builder.appName('read data through spark').getOrCreate()

### Create dataframes from cleaned data from T1

In [4]:
### There is a new Schemas folder which contains schemas for the cleaned csvs which were 
### created at the end of load_csv_into_schema.ipynb
CLEAN_DATA_FOLDER = './assets/cleanedDFsData/'
SCHEMAS_FOLDER = './cleanedDFsSchemas/'

### We load the neccessary csvs for T2 into the new schemas
paper_author_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}paper_author_ds.csv', f'{SCHEMAS_FOLDER}paper_authors.csv')
paper_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}papers_ds.csv', f'{SCHEMAS_FOLDER}paper.csv')
affiliation_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}affiliations_ds.csv', f'{SCHEMAS_FOLDER}affiliation.csv')
publication_venue_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}publication_venues_ds.csv', f'{SCHEMAS_FOLDER}publication_venues.csv')
research_interests_df = createDFFromFileAndSchema(spark, f'{CLEAN_DATA_FOLDER}research_interests_ds.csv', f'{SCHEMAS_FOLDER}research_interests.csv')


File path: ./assets/cleanedDFsData/research_interests_ds.csv, schema path: ./cleanedDFsSchemas/research_interests.csv
Types from schema: [('author_id', 'Integer'), ('research_interest', 'String')]
File path: ./assets/cleanedDFsData/papers_ds.csv, schema path: ./cleanedDFsSchemas/paper.csv
Types from schema: [('paper_id', 'Integer'), ('title', 'String'), ('year', 'Integer')]
File path: ./assets/cleanedDFsData/paper_author_ds.csv, schema path: ./cleanedDFsSchemas/paper_authors.csv
Types from schema: [('name', 'String'), ('paper_id', 'Integer'), ('author_id', 'Integer'), ('citation_count', 'Integer'), ('h_index', 'Integer'), ('paper_count', 'Integer')]
File path: ./assets/cleanedDFsData/affiliations_ds.csv, schema path: ./cleanedDFsSchemas/affiliation.csv
Types from schema: [('paper_id', 'Integer'), ('affiliation', 'String')]
File path: ./assets/cleanedDFsData/publication_venues_ds.csv, schema path: ./cleanedDFsSchemas/publication_venues.csv
Types from schema: [('paper_id', 'Integer'), ('

In [5]:
###  Join paper_autho_df with paper_df to retrieve information (year)
df_for_ml_with_papers = paper_author_df.join(paper_df, 'paper_id', 'inner')

In [6]:
df_for_ml_with_affiliations = df_for_ml_with_papers.join(affiliation_df, 'paper_id', 'left')

In [8]:
final_df_for_ml = df_for_ml_with_affiliations.join(research_interests_df, 'author_id', 'left').\
    drop('name', 'h_index', 'paper_count', 'citation_count', 'title')

### Create String Indexers and One Hot Encoders for the research interests String columns

In [10]:
# Create a StringIndexer
research_interests_indexer = StringIndexer(inputCol="research_interest", outputCol="research_interest_index")

# Create a OneHotEncoder
research_interests_encoder = OneHotEncoder(inputCol="research_interest_index", outputCol="research_interest_fact")

### Create String Indexers and One Hot Encoders for the affiliation String columns

In [11]:
    # Create a StringIndexer
affiliations_indexer = StringIndexer(inputCol="affiliation", outputCol="affiliation_index")

# Create a OneHotEncoder
affiliations_encoder = OneHotEncoder(inputCol="affiliation_index", outputCol="affiliation_fact")

In [12]:
vec_assembler = VectorAssembler( \
    inputCols=["year", "affiliation_fact", "research_interest_fact"], \
    outputCol="features" \
)

In [13]:
papers_pipe = Pipeline(
    stages=[research_interests_indexer,\
                research_interests_encoder, \
                affiliations_indexer, \
                affiliations_encoder, \
                vec_assembler \
           ])

In [14]:
# Fit and transform the data
piped_data = papers_pipe.fit(final_df_for_ml).transform(final_df_for_ml)

22/01/21 17:19:49 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from t-193.vc-graz.ac.at:63140 in 10000 milliseconds
22/01/21 17:20:06 WARN Executor: Issue communicating with driver in heartbeater]
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Ex

	at org.apache.spark.scheduler.AccumulableInfo.toString(AccumulableInfo.scala:41)
	at java.base/java.lang.String.valueOf(String.java:2951)
	at scala.collection.mutable.StringBuilder.append(StringBuilder.scala:203)
	at scala.collection.TraversableOnce$appender$1.apply(TraversableOnce.scala:419)
	at scala.collection.TraversableOnce$appender$1.apply(TraversableOnce.scala:410)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.TraversableOnce.addString(TraversableOnce.scala:424)
	at scala.collection.TraversableOnce.addString$(TraversableOnce.scala:407)
	at scala.collection.AbstractTraversable.addString(Traversable.scala:108)
	at scala.collection.

Py4JJavaError: An error occurred while calling o109.fit.
: org.apache.spark.SparkException: Job 10 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1115)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1113)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1113)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2615)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2515)
	at org.apache.spark.SparkContext.$anonfun$stop$12(SparkContext.scala:2086)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1442)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:2086)
	at org.apache.spark.SparkContext$$anon$

22/01/21 17:25:52 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 1.0 in stage 25.0 (TID 76)
java.lang.NullPointerException
	at org.apache.spark.scheduler.Task.$anonfun$run$2(Task.scala:152)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1442)
	at org.apache.spark.scheduler.Task.run(Task.scala:150)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:506)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1462)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:509)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
22/01/21 17:25:52 ERROR Executor: Exception in task 1.0 in stage 25.0 (TID 76): Java heap space
